ПАКЕТ SURPRISE

используйте данные MovieLens 1M
можно использовать любые модели из пакета
получите RMSE на тестовом сете 0.87 и ниже
Комментарий преподавателя :
В ДЗ на датасет 1М может не хватить RAM. Можно сделать на 100K. Качество RMSE предлагаю считать на основе CrossValidation (5 фолдов), а не отложенном датасете.

In [ ]:
import pandas as pd
import numpy as np

from tqdm import tqdm

In [ ]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

In [ ]:
movies_with_ratings = movies.merge(ratings, on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)

In [ ]:
movies_with_ratings.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483


In [ ]:
movies_with_ratings.title.value_counts()

Forrest Gump (1994)                      329
Shawshank Redemption, The (1994)         317
Pulp Fiction (1994)                      307
Silence of the Lambs, The (1991)         279
Matrix, The (1999)                       278
                                        ... 
We're Back! A Dinosaur's Story (1993)      1
American Hardcore (2006)                   1
Shanghai Surprise (1986)                   1
Let's Get Harry (1986)                     1
Andrew Dice Clay: Dice Rules (1991)        1
Name: title, Length: 9719, dtype: int64

In [ ]:
movies_with_ratings.userId.value_counts()

414    2698
599    2478
474    2108
448    1864
274    1346
       ... 
53       20
207      20
431      20
442      20
189      20
Name: userId, Length: 610, dtype: int64

In [ ]:
num_users = movies_with_ratings.userId.unique().shape[0]
num_users

610

In [ ]:
movies_with_ratings.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483


In [ ]:
movie_vector = {}

for movie, group in tqdm(movies_with_ratings.groupby('title')):
    movie_vector[movie] = np.zeros(num_users)
    
    for i in range(len(group.userId.values)):
        u = group.userId.values[i]
        r = group.rating.values[i]
        movie_vector[movie][int(u - 1)] = r

100%|██████████| 9719/9719 [00:03<00:00, 3179.61it/s]


In [ ]:
movie_vector['Toy Story (1995)']

array([4. , 0. , 0. , 0. , 4. , 0. , 4.5, 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 2.5, 0. , 4.5, 3.5, 4. , 0. , 3.5, 0. , 0. , 0. , 0. , 0. ,
       3. , 0. , 0. , 0. , 5. , 3. , 3. , 0. , 0. , 0. , 0. , 0. , 0. ,
       5. , 0. , 0. , 5. , 3. , 4. , 5. , 0. , 0. , 0. , 3. , 0. , 0. ,
       0. , 3. , 0. , 0. , 5. , 0. , 0. , 0. , 0. , 0. , 5. , 4. , 0. ,
       4. , 0. , 2.5, 0. , 0. , 5. , 0. , 4.5, 0. , 0. , 0.5, 0. , 4. ,
       0. , 0. , 0. , 2.5, 0. , 0. , 0. , 4. , 0. , 0. , 3. , 3. , 4. ,
       0. , 3. , 0. , 0. , 5. , 0. , 4.5, 0. , 0. , 0. , 0. , 4. , 0. ,
       0. , 0. , 4. , 0. , 0. , 0. , 0. , 3. , 0. , 0. , 0. , 0. , 0. ,
       0. , 3.5, 0. , 4. , 0. , 0. , 4. , 0. , 0. , 0. , 0. , 0. , 3. ,
       0. , 2. , 0. , 3. , 4. , 0. , 4. , 0. , 0. , 3. , 4. , 0. , 0. ,
       3.5, 5. , 0. , 0. , 0. , 0. , 0. , 5. , 0. , 2. , 0. , 3. , 4. ,
       0. , 0. , 4.5, 4. , 4. , 0. , 0. , 0. , 0. , 5. , 3.5, 0. , 4.5,
       0. , 5. , 0. , 0. , 0. , 0. , 0. , 5. , 4. , 4. , 0. , 0.

In [ ]:
from scipy.spatial.distance import cityblock, cosine, euclidean, hamming, jaccard

In [ ]:
my_fav_film = 'Fight Club (1999)'

titles = []
distances = []

for key in tqdm(movie_vector.keys()):
    if key == my_fav_film:
        continue
    
    titles.append(key)
    distances.append(cosine(movie_vector[my_fav_film], movie_vector[key]))

len(distances)

100%|██████████| 9719/9719 [00:00<00:00, 24212.52it/s]


9718

In [ ]:
best_indexes = np.argsort(distances)[:10]
best_indexes

array([5511, 5571,  420, 4736, 5203,  405, 6864, 5204, 4322, 4737])

In [ ]:
best_movies = [(titles[i], distances[i]) for i in best_indexes]

for m in best_movies:
    print(m)

('Matrix, The (1999)', 0.28606257328028306)
('Memento (2000)', 0.33040704494686757)
('American History X (1998)', 0.3509458374819995)
('Kill Bill: Vol. 1 (2003)', 0.360262413456623)
('Lord of the Rings: The Fellowship of the Ring, The (2001)', 0.36425580638414523)
('American Beauty (1999)', 0.3744505150386159)
('Pulp Fiction (1994)', 0.37678014788325576)
('Lord of the Rings: The Return of the King, The (2003)', 0.37798413607448667)
('Inception (2010)', 0.3845825728533704)
('Kill Bill: Vol. 2 (2004)', 0.38565615909903594)


### **Surprise**

In [ ]:
!pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from surprise import KNNWithMeans, KNNBasic
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split

import pandas as pd

In [ ]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

In [ ]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [ ]:
movies_with_ratings = movies.merge(ratings, on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)
movies_with_ratings.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483


In [ ]:
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

In [ ]:
dataset.head()

,uid,iid,rating
0,1,Toy Story (1995),4.0
1,5,Toy Story (1995),4.0
2,7,Toy Story (1995),4.5
3,15,Toy Story (1995),2.5
4,17,Toy Story (1995),4.5


In [ ]:
ratings.rating.min(), ratings.rating.max()

(0.5, 5.0)

In [ ]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(dataset, reader)
# data_1 = Dataset.load_from_df(movies_with_ratings, reader)

In [ ]:
trainset, testset = train_test_split(data, test_size=0.2, random_state=1)

In [ ]:
dataset['uid'].nunique(), dataset['iid'].nunique()

(610, 9719)

In [ ]:
algo = KNNWithMeans(k=50, sim_options={
    'name': 'cosine',
    'user_based': True  # compute  similarities between users
})
algo.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [ ]:
test_pred = algo.test(testset)
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8985


0.898482528669153

In [ ]:
test_pred[:3]

[Prediction(uid=542, iid='Kill Bill: Vol. 1 (2003)', r_ui=2.5, est=3.7592260572900007, details={'actual_k': 50, 'was_impossible': False}),
 Prediction(uid=469, iid='Bridge on the River Kwai, The (1957)', r_ui=4.0, est=4.168241951992395, details={'actual_k': 34, 'was_impossible': False}),
 Prediction(uid=317, iid='Scarface (1983)', r_ui=5.0, est=4.062599058241541, details={'actual_k': 49, 'was_impossible': False})]

In [ ]:
algo.predict(uid=2, iid='Fight Club (1999)')

Prediction(uid=2, iid='Fight Club (1999)', r_ui=None, est=4.474085079084162, details={'actual_k': 50, 'was_impossible': False})

In [ ]:
algo = KNNWithMeans(k=50, sim_options={
    'name': 'cosine',
    'user_based': False  # compute similarities between items
})
algo.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [ ]:
test_pred = algo.test(testset)

In [ ]:
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8982


0.898181158065249

In [ ]:
new_pred = algo.predict(uid=2, iid='Fight Club (1999)')
new_pred

Prediction(uid=2, iid='Fight Club (1999)', r_ui=None, est=4.2619917274785735, details={'actual_k': 24, 'was_impossible': False})

In [ ]:
# accuracy.rmse(new_pred, verbose=True)

TypeError: ignored

In [ ]:
# new_pred = algo.predict(uid=20000, iid='Fight Club (1999)')
# new_pred

In [ ]:
new_pred = algo.predict(uid=2, iid='unkown')
new_pred

Prediction(uid=2, iid='unkown', r_ui=None, est=3.498958694897605, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'})

In [ ]:
trainset.global_mean

3.498958694897605

In [ ]:
def generate_recommendation(uid, model, dataset, thresh=4.5, amount=5):
    all_titles = list(dataset['iid'].values)
    users_seen_titles = dataset[dataset['uid'] == uid]['iid']
    titles = np.array(list(set(all_titles) - set(users_seen_titles)))

    np.random.shuffle(titles)
    
    rec_list = []
    for title in titles:
        review_prediction = model.predict(uid=uid, iid=title)
        rating = review_prediction.est

        if rating >= thresh:
            rec_list.append((title, round(rating, 2)))
            
            if len(rec_list) >= amount:
                return rec_list

In [ ]:
generate_recommendation(2, algo, dataset)

[('Rififi (Du rififi chez les hommes) (1955)', 4.8),
 ('Indignation (2016)', 5.0),
 ('The Magician (1958)', 4.68),
 ('A Detective Story (2003)', 4.82),
 ('Love and Pigeons (1985)', 5.0)]

Кросс-валидация нормального человека:

In [ ]:
# from sklearn.model_selection import cross_val_score
# scores = cross_val_score(model,X_train, Y_train , cv = 5)
# scores.mean(), scores

In [ ]:
# from sklearn.model_selection import KFold
# kf = KFold(n_folds = 5, random_state=7)
# scores = cross_val_score(model, X_train,Y_train, cv = kf)
# scores.mean(). scores

In [ ]:
# from sklearn.model_selection import StratifiedKFold
# skf = StratifiedKFold(n_folds=5)
# scores = cross_val_score(model, X_train, Y_train, cv = skf)
# scores.mean(), scores

Кросс-валидация курильщика =)

In [ ]:
# 1 кросс-валидация на knn
# scores = cross_val_score(algo, trainset, algo.test(trainset), cv = skf)
# scores.mean(), scores

In [ ]:
from surprise.model_selection.split import KFold
from surprise.model_selection.validation import cross_validate

# 1. кросс-валидациа алгоритма knn из лекции:

# на scores = cross_validate(algo=algo, data=testset,cv=kf)
# ругается:
# AttributeError: 'list' object has no attribute 'raw_ratings'
# погуглила, пишут со всей датой ипользовать, так что технически на одном тесте
# не полуситься проверить

kf = KFold(n_splits=5,random_state=21)
scores = cross_validate(algo=algo, data=data,cv=kf)


Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.


In [ ]:
scores['test_rmse'], scores['test_rmse'].mean()

(array([0.90135358, 0.90952593, 0.89215762, 0.90486007, 0.90461681]),
 0.9025027998620349)

In [ ]:
scores

{'test_rmse': array([0.90135358, 0.90952593, 0.89215762, 0.90486007, 0.90461681]),
 'test_mae': array([0.69102055, 0.69352003, 0.68282578, 0.69042998, 0.68804853]),
 'fit_time': (7.675583600997925,
  7.800671577453613,
  7.430624723434448,
  7.8343446254730225,
  7.622232437133789),
 'test_time': (10.059746503829956,
  10.429024696350098,
  10.22658085823059,
  9.510959148406982,
  10.24720311164856)}

In [ ]:
# 2. попробую 2 алгоритма:
# matrix_factorization.SVD и SlopeOne
from surprise.prediction_algorithms.matrix_factorization import SVD
from surprise.prediction_algorithms.slope_one import SlopeOne

algo_svd = SVD()
algo_svd.fit(trainset)

algo_one = SlopeOne()
algo_one.fit(trainset)

In [ ]:
scores = cross_validate(algo=algo_svd, data=data,cv=kf)
scores['test_rmse'], scores['test_rmse'].mean()
# 8730459102874191 - цель не достигнута совсем чуть-чуть

In [ ]:
scores = cross_validate(algo=algo_one, data=data,cv=kf)
scores['test_rmse'], scores['test_rmse'].mean()
# почти как на старте с кнн

In [ ]:
# по ссылке из блокнота лекции было написано, что он лучший
from surprise.prediction_algorithms.matrix_factorization import SVDpp
algo_svd_pp = SVDpp()
algo_svd_pp.fit(trainset)
scores = cross_validate(algo=algo_svd_pp, data=data,cv=kf)
# бинго! не подвёл  0.8618396005138443)  0.8613547781131483)
# попробую потыкать параметы
scores['test_rmse'], scores['test_rmse'].mean()

(array([0.8610488 , 0.86607388, 0.85391755, 0.86369782, 0.86203584]),
 0.8613547781131483)

In [ ]:
scores['test_rmse'], scores['test_rmse'].mean()

(array([0.8610488 , 0.86607388, 0.85391755, 0.86369782, 0.86203584]),
 0.8613547781131483)

In [ ]:
# а вот интерестно - увеличение параметров не поднимет ещё точность?
# увеличение числа параметров привело к ухудшению модели
# логично попробывать < 100 factors!
# algo_svd_pp = SVDpp(n_factors =110, verbose=False, cache_ratings=True)
# algo_svd_pp.fit(trainset)
# scores = cross_validate(algo=algo_svd_pp, data=data,cv=kf)
# scores['test_rmse'], scores['test_rmse'].mean()

KeyboardInterrupt: ignored

In [ ]:
# попробую ещё улучшить подбором по сетке, надоело смотреть как долго он просто один результат считает
from surprise.model_selection.search import GridSearchCV #, RandomizedSearch - не нашла рандомную, с авто улучшением буду с тем что есть работать
# попытка 1:
# param_grid = {"n_epochs": [5, 10], "lr_all": [0.002, 0.005], "reg_all": [0.4, 0.6]}
# {'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4} = это то что он выбрал == best_params
# попытка 2:
# -----'n_epochs': [10,20],, 'n_factors': [80,100] = оооч долго
# param_grid = { 'lr_all': [0.004, 0.005],
#               'reg_all': [0.4, 0.5]}
              # {'lr_all': 0.005, 'reg_all': 0.4} == best
# попытка 3
# param_grid = {'lr_all': [0.005,0.006], } 
# {'lr_all': 0.005, 'reg_all': 0.4} == best

# попытка 4
# лучшие
# lr_all': 0.005    'reg_all': 0.4, 'n_factors': [80
# from surprise.prediction_algorithms.matrix_factorization import SVDpp
# param_grid = {'lr_all': [0.005,], 'reg_all': [0.4,], 'n_factors': [60,]}
# gs = GridSearchCV(SVDpp, param_grid, measures=["rmse",], cv=kf, n_jobs=-1)


In [ ]:
# gs.fit(data)

In [ ]:
# print(gs.best_score["rmse"])
# print(gs.best_params["rmse"])

0.8827735564205753


In [ ]:
from surprise.prediction_algorithms.matrix_factorization import SVDpp
# algo = gs.best_estimator["rmse"]
# проверю на травных условиях, как и остальные
algo_svd_pp = SVDpp(lr_all = 0.005, reg_all = 0.4, n_factors =60, verbose=False, cache_ratings=True)
algo_svd_pp.fit(trainset)
scores = cross_validate(algo=algo_svd_pp, data=data,cv=kf)
scores['test_rmse'], scores['test_rmse'].mean()

from surprise.prediction_algorithms.matrix_factorization import SVDpp
algo_svd_pp = SVDpp()
algo_svd_pp.fit(trainset)
scores = cross_validate(algo=algo_svd_pp, data=data,cv=kf)
# бинго! не подвёл  0.8618396005138443)
# на  0.000945855563664777 меньше обычногосвдпп
scores['test_rmse'], scores['test_rmse'].mean()

(array([0.86008059, 0.86752488, 0.85145706, 0.8627509 , 0.86023119]),
 0.8604089225494835)

In [ ]:
#svdpp_standart - svdpp_optimal
0.8613547781131483 - 0.8604089225494835

0.000945855563664777

https://programmer.ink/think/movie-recommendation-using-surprise-package.html


https://colab.research.google.com/github/singhsidhukuldeep/Recommendation-System/blob/master/Building_Recommender_System_with_Surprise.ipynb


https://www.programmersought.com/article/8414465116/

https://russianblogs.com/article/4267703864/


https://surprise.readthedocs.io/en/stable/getting_started.html#tuning-algorithm-parameters


https://surprise.readthedocs.io/en/stable/getting_started.html#tuning-algorithm-parameters


https://surpriselib.com/


https://colab.research.google.com/drive/1kJy_PTavRetvCOQLYsDdMQKmVLzS69Zy?usp=sharing#scrollTo=HY19CFiBRr90